# Intrinsic Dimension Estimators for Uncovering the Phase Space Dimensionality of Dynamical Systems

We compare the following eight intrinsic dimension estimators:
- Grassberger and Procaccia's Correlation Integral (`CorrInt`)
- Farahmand et al.'s Manifold-Adaptive Dimension estimation Algorithm (`MADA`)
- Levina and Bickel's and Haro et al.'s maximum likelihood-based estimators (`Levina_Bickel` and `MLE`)
- Rozza et al.'s Minimum Neighbour Distance-Maximum Likelihood estimators (`MiND_ML`)
- Facco et al.'s Two-Nearest Neighbour estimator (`TwoNN`)
- Amsaleg et al.'s method of moments estimator (`MOM`)
- Hein and Audibert's U-statistic-based estimator (`Hein`)

Requirements include `MATLAB`/`matlab.engine` and `sklearn`,`skdim`

In [5]:
import skdim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import matlab.engine

from sklearn.neighbors import NearestNeighbors

### Load files

Files should be stored as `.npy` files in a `latent` folder as `./latent/system_name_root_number.npy`

In [6]:
def file_loader(system, root):
    path = './latent/' + system + '_' + str(root) + '.npy' 
    return np.load(path)

### ID estimator
From Boyuan Chen et al. : https://github.com/BoyuanChen/neural-state-variables/tree/main/analysis

In [7]:
class ID_Estimator:
    def __init__(self, method='Levina_Bickel'):
        self.all_methods = ['Levina_Bickel', 'MiND_ML', 'MiND_KL', 'Hein', 'CD']
        self.set_method(method)
    
    def set_method(self, method='Levina_Bickel'):
        if method not in self.all_methods:
            assert False, 'Unknown method!'
        else:
            self.method = method
        
    def fit(self, X, k_list=20, n_jobs=4):
        if self.method in ['Hein', 'CD']:
            dim_Hein, dim_CD = Hein_CD(X)
            return dim_Hein if self.method=='Hein' else dim_CD
        else:
            if np.isscalar(k_list):
                k_list = np.array([k_list])
            else:
                k_list = np.array(k_list)
            kmax = np.max(k_list) + 2
            dists, inds = kNN(X, kmax, n_jobs)
            dims = []
            for k in k_list:
                if self.method == 'Levina_Bickel':
                    dims.append(Levina_Bickel(X, dists, k))
                elif self.method == 'MiND_ML':
                    dims.append(MiND_ML(X, dists, k))
                elif self.method == 'MiND_KL':
                    dims.append(MiND_KL(X, dists, k))
                else:
                    pass
            if len(dims) == 1:
                return dims[0]
            else:
                return np.array(dims)
    
    def fit_all_methods(self, X, k_list=[20], n_jobs=4):
        k_list = np.array(k_list)
        kmax = np.max(k_list) + 2
        dists, inds = kNN(X, kmax, n_jobs)
        dim_all_methods = {method:[] for method in self.all_methods}
        dim_all_methods['Hein'], dim_all_methods['CD'] = Hein_CD(X)
        for k in k_list:
            dim_all_methods['Levina_Bickel'].append(Levina_Bickel(X, dists, k))
            dim_all_methods['MiND_ML'].append(MiND_ML(X, dists, k))
            dim_all_methods['MiND_KL'].append(MiND_KL(X, dists, k))
        for method in self.all_methods:
            dim_all_methods[method] = np.array(dim_all_methods[method])
        return dim_all_methods

### Methods
From Boyuan Chen et al. : https://github.com/BoyuanChen/neural-state-variables/tree/main/analysis

In [8]:
def kNN(X, n_neighbors, n_jobs):
    neigh = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=n_jobs).fit(X)
    dists, inds = neigh.kneighbors(X)
    return dists, inds


def Levina_Bickel(X, dists, k):
    m = np.log(dists[:, k:k+1] / dists[:, 1:k])
    m = (k-2) / np.sum(m, axis=1)
    dim = np.mean(m)
    return dim


def start_matlab_engine():
    import matlab.engine
    eng = matlab.engine.start_matlab()
    #eng.addpath(os.path.join(os.path.split(__file__)[0], 'matlab_codes'))
    return eng


def MiND_ML(X, dists, k):
    import matlab.engine
    eng = start_matlab_engine()
    X_mat = matlab.double(X.T.tolist())
    dists_mat = matlab.double(dists[:, :k+2].tolist())
    dim = eng.MiND_ML(X_mat, 'dists', dists_mat, 'normalized', False, 'optimize', True)
    return dim


def MiND_KL(X, dists, k, maxDim=30):
    import matlab.engine
    eng = start_matlab_engine()
    X_mat = matlab.double(X.T.tolist())
    dists_mat = matlab.double(dists[:, :k+2].tolist())
    dim = eng.MiND_KL(X_mat, 'k', matlab.double([k]), 'maxDim', matlab.double([maxDim]), 'dists', dists_mat, 'normalized', False, nargout=1)
    return dim


def DANCo(X, dists, inds, k, maxDim=30):
    import matlab.engine
    eng = start_matlab_engine()
    X_mat = matlab.double(X.T.tolist())
    dists_mat = matlab.double(dists[:, :k+2].tolist())
    inds_mat = matlab.int32((inds[:, :k+2]+1).tolist())  # fit Matlab indices
    dim = eng.DANCo(X_mat, 'k', matlab.double([k]), 'maxDim', matlab.double([maxDim]), 'fractal', True, 'dists', dists_mat, 'inds', inds_mat, 'normalized', False, nargout=1)
    return dim


def Hein_CD(X):
    import matlab.engine
    eng = start_matlab_engine()
    X_mat = matlab.double(X.T.tolist())
    dim = eng.GetDim(X_mat, nargout=1)
    dim = np.array(dim)[0]
    return dim[0], dim[1]

### Estimators `Levina_Bickel`, `Hein`
From Boyuan Chen et al. : https://github.com/BoyuanChen/neural-state-variables/tree/main/analysis

In [ ]:
# Systems: reaction_diffusion, single_pendulum, double_pendulum, swingstick_non_magnetic, elastic_pendulum
# Methods: Levina_Bickel, Hein

system = 'single_pendulum'
method = 'Levina_Bickel'

estimator = ID_Estimator(method=method)

mean = 0

for root in [1,2,3]:
    latent = file_loader(system=system, root=root)
    # Neighbors list as in original code/paper
    k_list = (latent.shape[0] * np.linspace(0.008, 0.016, 5)).astype('int')
    estimation = np.mean(estimator.fit(latent, k_list))
    mean += estimation
print(system, method, round(mean/3., 2))

### Estimators `MiND_ML`, `MLE`, `TwoNN`, `MADA`, `MOM`, `CorrInt`

In [ ]:
# Systems: reaction_diffusion, single_pendulum, double_pendulum, swingstick_non_magnetic, elastic_pendulum
# Methods: skdim.id.MiND_ML(), skdim.id.MLE(), skdim.id.TwoNN(), skdim.id.MADA(), skdim.id.MOM(), skdim.id.CorrInt()

system = 'single_pendulum'
method = skdim.id.MLE()

mean = 0
for root in [1,2,3]:
    latent = file_loader(system=system, root=root)
    method_fited = method.fit(latent)
    mean += method_fited.dimension_
print(system, method, round(mean/3., 2))